In [12]:
# import libraries and navigate to directory
%cd '/home/uzumochi/eigenjuno/DATA/RAW/PERIJOVE-20'
from stitch import *
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os, threading, time, multiprocessing, sys, json, math, zipfile, fnmatch, shutil, pickle, logging

/home/uzumochi/eigenjuno/DATA/RAW/PERIJOVE-20


In [ ]:
# handle occasional spice exceptions - code still runs fine
f_err = sys.stderr = open('errors.txt', 'w')
get_ipython().log.handlers[0].stream = f_err
get_ipython().log.setLevel(logging.INFO)

In [13]:
# unzip junocam website data - just drop zip files in perijove directory and run this block!
for file in sorted(os.listdir('.')):
    meta_num = file.split('-')[0]
    if fnmatch.fnmatch(file, '*-Data.zip') and not os.path.isfile(meta_num + '-Metadata.json'):
        print('Unzipped', meta_num, 'image and data')
        with zipfile.ZipFile(file, 'r') as zip_ref:
            names = zip_ref.namelist()
            for n in names:
                zip_ref.extract(n)
        with zipfile.ZipFile(meta_num + '-ImageSet.zip', 'r') as zip_ref:
            names = zip_ref.namelist()
            for n in names:
                if 'raw' in n:
                    zip_ref.extract(n)
for file in sorted(os.listdir('.')):
    if fnmatch.fnmatch(file, '*.zip') or fnmatch.fnmatch(file, '*Zone.Identifier'):
        os.remove(file)

if os.path.isdir('DataSet'):
    for file in sorted(os.listdir('DataSet')):
        shutil.move('DataSet/' + file, file)
    os.rmdir('DataSet')

if os.path.isdir('ImageSet'):
    for file in sorted(os.listdir('ImageSet')):
        shutil.move('ImageSet/' + file, file)
    os.rmdir('ImageSet')

Unzipped 6978 image and data
Unzipped 6980 image and data
Unzipped 6983 image and data
Unzipped 6984 image and data
Unzipped 6991 image and data
Unzipped 6993 image and data
Unzipped 6994 image and data


In [14]:
# stitch function for thread, taken largely from the reddit post linked in repo desc
def stitch(meta_num):
    im_info = meta_num + '-Metadata.json'
    with open(im_info, 'r') as json_file:
        im_info_dir = json.load(json_file)
        image = im_info_dir['FILE_NAME']
        image_time = im_info_dir['IMAGE_TIME']
    
    img = Image.open(image)
    im_ar = np.array(img)
    
    s1, s2 = im_ar.shape

    start_time = im_info_dir["START_TIME"]
    frame_delay = float(im_info_dir["INTERFRAME_DELAY"].split()[0])+0.001
    
    start_correction, frame_delay = correct_image_start_time_and_frame_delay(im_ar, start_time, frame_delay)
    
    framelets = generate_framelets(im_ar, start_time, start_correction, frame_delay) # im_ar.astype(np.int) ** 2

    cam_pos, cam_orient = get_junocam_jupiter_rel_pos_orient(start_time, start_correction + 17 * frame_delay)

    y, x = np.mgrid[-1024:1024,-1024:1024]
    x += 260
    y += 95
    rays = np.concatenate([x[...,None], y[...,None], np.ones(((2048),(2048),1))*fl[0]], axis=-1)
    rays = rays.dot(cam_orient)

    surface_raster, surface_mask = project_onto_jupiter_surf(cam_pos, rays)
    
    np.save(meta_num + '-Raster', surface_raster)
    
    colors = np.zeros(((2048),(2048),3))
    color_counts = np.zeros(((2048),(2048),3))

    for k,framelet in enumerate(framelets):
        print(str(meta_num) + ': processing framelet {} of {}..'.format(k+1, len(framelets)))
        col = framelet.color
        brightnesses, valid_map = framelet.get_pixel_val_at_surf_point(surface_raster)
        colors[...,2-col] += brightnesses
        color_counts[...,2-col] += valid_map

    colors /= np.maximum(color_counts, 1)
    colors *= 255 / np.max(colors)

    colors = colors.astype(np.uint8)

    new_img = Image.fromarray(colors)
    new_img.save(str(meta_num) + '-Stitched.png')

In [15]:
#begin multithreaded execution - this will process all images in a directory, assuming they are from the same perijove
img_nums = []
for file in os.listdir('.'):
    if file.endswith('.json') and not os.path.isfile(file.split('-')[0] + '-Stitched.png'):
        img_nums.append(file.split('-')[0])
print_str = 'Processing the following images: '
for im in img_nums:
    print_str += (im + ', ')
print_str = print_str[:-2]
print(print_str)

num_cpus = multiprocessing.cpu_count() - 1
curr = 0

while curr < len(img_nums):
    threads = []
    num_threads = num_cpus if num_cpus < len(img_nums) else len(img_nums)
    print('Running ' + str(num_threads) + ' threads')
    for t in range(num_threads):
        try:
            threads.append(threading.Thread(target = stitch, args = (str(img_nums[curr]), )))
        except IndexError:
            print('\nStarting new round of threads')
        threads[t].start()
        curr += 1
    for t in threads:
        t.join()

print("End stitching threads")

Processing the following images: 6978, 6980, 6983, 6984, 6991, 6993, 6994
Running 7 threads
6993: processing framelet 1 of 42..
6994: processing framelet 1 of 42..
6991: processing framelet 1 of 48..
6984: processing framelet 1 of 60..
6983: processing framelet 1 of 60..
6978: processing framelet 1 of 90..
6980: processing framelet 1 of 72..
6994: processing framelet 2 of 42..
6993: processing framelet 2 of 42..
6991: processing framelet 2 of 48..
6984: processing framelet 2 of 60..
6983: processing framelet 2 of 60..
6978: processing framelet 2 of 90..
6994: processing framelet 3 of 42..
6980: processing framelet 2 of 72..
6993: processing framelet 3 of 42..
6991: processing framelet 3 of 48..
6984: processing framelet 3 of 60..
6983: processing framelet 3 of 60..
6994: processing framelet 4 of 42..
6993: processing framelet 4 of 42..
6978: processing framelet 3 of 90..
6980: processing framelet 3 of 72..
6991: processing framelet 4 of 48..
6984: processing framelet 4 of 60..
6983: pr

6984: processing framelet 32 of 60..
6994: processing framelet 37 of 42..
6983: processing framelet 32 of 60..
6991: processing framelet 35 of 48..
6993: processing framelet 36 of 42..
6978: processing framelet 28 of 90..
6980: processing framelet 28 of 72..
6984: processing framelet 33 of 60..
6994: processing framelet 38 of 42..
6983: processing framelet 33 of 60..
6991: processing framelet 36 of 48..
6993: processing framelet 37 of 42..
6994: processing framelet 39 of 42..
6984: processing framelet 34 of 60..
6978: processing framelet 29 of 90..
6980: processing framelet 29 of 72..
6983: processing framelet 34 of 60..
6991: processing framelet 37 of 48..
6993: processing framelet 38 of 42..
6994: processing framelet 40 of 42..
6984: processing framelet 35 of 60..
6991: processing framelet 38 of 48..
6978: processing framelet 30 of 90..
6980: processing framelet 30 of 72..
6983: processing framelet 35 of 60..
6993: processing framelet 39 of 42..
6994: processing framelet 41 of 42..
6

/tmp/ipykernel_31248/3864622598.py:44: RuntimeWarning: divide by zero encountered in double_scalars
  colors *= 255 / np.max(colors)
/tmp/ipykernel_31248/3864622598.py:44: RuntimeWarning: invalid value encountered in multiply
  colors *= 255 / np.max(colors)


6983: processing framelet 37 of 60..
6984: processing framelet 38 of 60..
6980: processing framelet 32 of 72..
6978: processing framelet 32 of 90..
6991: processing framelet 41 of 48..
6993: processing framelet 42 of 42..
6983: processing framelet 38 of 60..
6984: processing framelet 39 of 60..
6991: processing framelet 42 of 48..
6980: processing framelet 33 of 72..
6978: processing framelet 33 of 90..
6983: processing framelet 39 of 60..
6984: processing framelet 40 of 60..
6991: processing framelet 43 of 48..
6980: processing framelet 34 of 72..
6978: processing framelet 34 of 90..
6984: processing framelet 41 of 60..
6983: processing framelet 40 of 60..
6991: processing framelet 44 of 48..
6980: processing framelet 35 of 72..
6984: processing framelet 42 of 60..
6991: processing framelet 45 of 48..
6983: processing framelet 41 of 60..
6978: processing framelet 35 of 90..
6980: processing framelet 36 of 72..
6991: processing framelet 46 of 48..
6984: processing framelet 43 of 60..
6

In [16]:
# copy processed images to testing directory
for file in sorted(os.listdir('.')):
    if fnmatch.fnmatch(file, '*-Metadata.json') or fnmatch.fnmatch(file, '*-Raster.npy') \
        or fnmatch.fnmatch(file, '*-Stitched.png'):
        shutil.copy(file, '../../TEST/' + file)